# Get Projections

<br/>

<pre>
model name:            imagenette_128_resnet18_model.pth
network architecture:  resnet18
dataset:               imagenette training set
image size:            128x128 (resized beforehand)
</pre>

<br/>

We want to test our Out-of-Distribution (OoD) detection method __Layer-wise Activation Cluster Analysis (LACA)__ on a dataset that is more complex than the MNIST, SVHN or the CIFAR-10 dataset which have been used so far. We chose the [Imagenette dataset](https://github.com/fastai/imagenette) as it contains images showing more complex scenes. The [Imagenette dataset](https://github.com/fastai/imagenette) is a subset of 10 classes of the [ImageNet dataset](https://www.image-net.org/). 

The first step of our OoD detection method is executed before inference. Here we measure in-distribution statistics from the training data and OoD statistics from the calibration data. Both kind of statistics are necessary to calculate the credibility of a test sample at inference. 

After fetching (see __01_fetch_activations_imagenette_128_resnet18.ipynb__) and vectorizing the activations (see __02_vectorize_activations_imagenette_128_resnet18.ipynb__) from the data samples we get the 2D projections of the activations. This is necessary because to obtain the statistics we need to find clusters in the data. However, finding clusters in high-dimensional spaces is difficult. Thus, we get the projections.

<br/>

_Sources:_
* [Imagenette dataset](https://github.com/fastai/imagenette)
* [Deep kNN paper](https://arxiv.org/abs/1803.04765)
* [Deep kNN sample code](https://github.com/cleverhans-lab/cleverhans/blob/master/cleverhans_v3.1.0/cleverhans/model_zoo/deep_k_nearest_neighbors/dknn.py)
* [Deep kNN sample code (PyTorch)](https://github.com/bam098/deep_knn/blob/master/dknn_mnist.ipynb)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.functional import adaptive_avg_pool2d
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms, models, datasets
import sklearn
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import metrics
import skimage
from skimage.measure import block_reduce
from umap import UMAP
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle
import numpy as np
import platform
from pathlib import Path
import random
import warnings
import pprint
from collections import Counter

sns.set()
sns.set_context("notebook", font_scale=1.1)
sns.set_style("ticks")

print('python version:      {}'.format(platform.python_version()))
print('torch version:       {}'.format(torch.__version__))
print('torchvision version: {}'.format(torchvision.__version__))
print('sklearn version:     {}'.format(sklearn.__version__))
print('skimage version:     {}'.format(skimage.__version__))
print('numpy version:       {}'.format(np.__version__))
print('matplotlib version:  {}'.format(matplotlib.__version__))
print('seaborn version:     {}'.format(sns.__version__))
print('pandas version:      {}'.format(pd.__version__))
print('pickle version:      {}'.format(pickle.format_version))

use_cuda = torch.cuda.is_available()
print('CUDA available:      {}'.format(use_cuda))
print('cuDNN enabled:       {}'.format(torch.backends.cudnn.enabled))
print('num gpus:            {}'.format(torch.cuda.device_count()))

if use_cuda:
    print('gpu:                 {}'.format(torch.cuda.get_device_name(0)))

    print()
    print('------------------------- CUDA -------------------------')
    ! nvcc --version

python version:      3.6.9
torch version:       1.7.0
torchvision version: 0.8.1
sklearn version:     0.23.2
skimage version:     0.17.2
numpy version:       1.19.5
matplotlib version:  3.2.2
seaborn version:     0.11.0
pandas version:      1.1.4
pickle version:      4.0
CUDA available:      False
cuDNN enabled:       True
num gpus:            0


We set the seed values to obtain reproducible results. For more information how to set seed values in Python and Pytorch see the [Pytorch documentation](https://pytorch.org/docs/1.7.0/notes/randomness.html?highlight=repro).

In [2]:
seed = 0
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.set_deterministic(True)

## Parameters

In [3]:
# Activations
img_size          = 128                                                             # Image size
base_act_folder   = Path('/Users/lehmann/research/laca3/activations/imagenette')    # Base activations folder
afname_string     = 'imagenette_{}_resnet18_acts'.format(img_size)                  # Activations file name
acts_path         = base_act_folder/afname_string                                   # Activations path


# Projections
base_projs_folder = Path('/Users/lehmann/research/laca3/projections/imagenette')    # Base projection folder
pfname_string     = 'imagenette_{}_resnet18_projs'.format(img_size)                 # Projections file name
projs_path        = base_projs_folder/pfname_string                                 # Activations path
layer_names       = [                                                               # List of layer names 
    'relu',
    'maxpool',
    'layer1-0',
    'layer1-1',
    'layer2-0',
    'layer2-1',
    'layer3-0',
    'layer3-1',
    'layer4-0',
    'layer4-1',
    'avgpool'
]

## Define Function for Getting the Projections

In [4]:
def get_projections(dataset_name, reduce, ext_scaler=None, ext_reducer=None):

    for layer_name in layer_names:
        print('## Computing Projections for Layer {}'.format(layer_name))
        
        # Load activations
        fname = str(acts_path) + '_{}_{}_vectors.pkl'.format(dataset_name, layer_name)
        with open(fname, 'rb') as pickle_file:
            loaded_activations = pickle.load(pickle_file)
        
        # Normalize activations        
        if ext_scaler:
            print('- external scaler is used')
            scaler = ext_scaler[layer_name]
        else:
            scaler = preprocessing.StandardScaler()       
            scaler.fit(loaded_activations['activations'])
            
        layer_activations_norm = scaler.transform(loaded_activations['activations'])
        
        print('- activations normalized: {}'.format(layer_activations_norm.shape))
        
        # Reduce activations
        if ext_reducer:
            print('- external reducer is used')
            layer_projections, reducer = reduce(layer_activations_norm, ext_reducer[layer_name])
        else:
            layer_projections, reducer = reduce(layer_activations_norm)
            
        print('- activations reduced: {}'.format(layer_projections.shape))
        
        # Save projections and reducer
        projections = {}
        projections['projections'] = layer_projections
        projections['targets'] = loaded_activations['targets']
        projections['scaler'] = scaler
        projections['reducer'] = reducer
        
        fname = str(projs_path) + '_{}_{}.pkl'.format(dataset_name, layer_name)
        with open(fname, 'wb') as pickle_file:
            pickle.dump(projections, pickle_file, protocol=4)

        print("done!")        
        print()

def pca_umap_reduce(activations, ext_reducer=None):
    # An external reducer is used
    if ext_reducer:
        proj_temp = activations
        for reducer in ext_reducer:
            proj_temp = reducer.transform(proj_temp)
            
        return proj_temp, ext_reducer
    
    # No external reducer available, we must fit a reducer
    umap_reducer = UMAP(n_components=2, metric='cosine', n_neighbors=15, min_dist=0.0)
    
    if activations.shape[1] > 50:
        pca_reducer = PCA(n_components=50)
        pca_reducer.fit(activations)
        proj_temp = pca_reducer.transform(activations)
        umap_reducer.fit(proj_temp)
        return umap_reducer.transform(proj_temp), [pca_reducer, umap_reducer]
    else:
        umap_reducer.fit(activations)
        return umap_reducer.transform(activations), [umap_reducer]
    

## Getting Projections

In [5]:
trainset_name = 'trainset'
get_projections(trainset_name, pca_umap_reduce)

## Computing Projections for Layer relu
- activations normalized: (9469, 262144)
- activations reduced: (9469, 2)
done!

## Computing Projections for Layer maxpool
- activations normalized: (9469, 65536)
- activations reduced: (9469, 2)
done!

## Computing Projections for Layer layer1-0
- activations normalized: (9469, 65536)
- activations reduced: (9469, 2)
done!

## Computing Projections for Layer layer1-1
- activations normalized: (9469, 65536)
- activations reduced: (9469, 2)
done!

## Computing Projections for Layer layer2-0
- activations normalized: (9469, 32768)
- activations reduced: (9469, 2)
done!

## Computing Projections for Layer layer2-1
- activations normalized: (9469, 32768)
- activations reduced: (9469, 2)
done!

## Computing Projections for Layer layer3-0
- activations normalized: (9469, 16384)
- activations reduced: (9469, 2)
done!

## Computing Projections for Layer layer3-1
- activations normalized: (9469, 16384)
- activations reduced: (9469, 2)
done!

## Computing

In [6]:
loaded_train_projections = {}

for layer_name in layer_names:
    print('## layer {}'.format(layer_name))
        
    fname = str(projs_path) + '_{}_{}.pkl'.format(trainset_name, layer_name)
    with open(fname, 'rb') as pickle_file:
        loaded_train_projections = pickle.load(pickle_file)
    
    print('activations: {}, targets: {}'.format(
        loaded_train_projections['projections'].shape, loaded_train_projections['targets'].shape
    ))  
    print('scaler:      {}'.format(loaded_train_projections['scaler']))     
    print('reducer:     {}'.format(loaded_train_projections['reducer']))
    print()

## layer relu
activations: (9469, 2), targets: (9469,)
scaler:      StandardScaler()
reducer:     [PCA(n_components=50), UMAP(angular_rp_forest=True, metric='cosine', min_dist=0.0)]

## layer maxpool
activations: (9469, 2), targets: (9469,)
scaler:      StandardScaler()
reducer:     [PCA(n_components=50), UMAP(angular_rp_forest=True, metric='cosine', min_dist=0.0)]

## layer layer1-0
activations: (9469, 2), targets: (9469,)
scaler:      StandardScaler()
reducer:     [PCA(n_components=50), UMAP(angular_rp_forest=True, metric='cosine', min_dist=0.0)]

## layer layer1-1
activations: (9469, 2), targets: (9469,)
scaler:      StandardScaler()
reducer:     [PCA(n_components=50), UMAP(angular_rp_forest=True, metric='cosine', min_dist=0.0)]

## layer layer2-0
activations: (9469, 2), targets: (9469,)
scaler:      StandardScaler()
reducer:     [PCA(n_components=50), UMAP(angular_rp_forest=True, metric='cosine', min_dist=0.0)]

## layer layer2-1
activations: (9469, 2), targets: (9469,)
scaler:    